This notebook maps names that are variations of each other eg 'mark jones' maps to 'm jones'

And saves the results to a postgres database

In [1]:
import pandas as pd

import json

from authorRe import dmatch

import authorUtils as util

import sqlUtils as q

from sqlalchemy import create_engine

from pprint import pprint  # pretty-printery

from IPython.display import clear_output 

# read and process parsed author string

In [2]:
df = pd.read_csv('author_string_names.csv', keep_default_na=False)

In [3]:
df.journal_format.unique()

array(['only_names', 'names_initials_names', 'initials_names',
       'names_comma_names', 'names_initials', 'error.blank',
       'names_comma_names_initials', 'error.unknown_format',
       'error.no_spaces', 'error.illegal_character', 'error.list',
       'error.only_initials', 'unknown.names_comma_initials_names',
       'error.too_long'], dtype=object)

In [4]:
df = df.loc[(~df.journal_format.str.match('error'))&(~df.journal_format.str.match('unknown'))]

In [5]:
df.journal_format.unique()

array(['only_names', 'names_initials_names', 'initials_names',
       'names_comma_names', 'names_initials',
       'names_comma_names_initials'], dtype=object)

# add indexes on first initial and last name

In [6]:
# add indexes on first initial and last name to speed up matching - author string id required to make index unique

df = df.sort_values(['first_initial', 'last_name', 'forenames', 'initials', 'author_string_id'], ascending=False).reset_index(drop=True)

df['indexi'] = df.first_initial
df['indexl'] = df.last_name
df['indexas'] = df.author_string_id

df = df.set_index(keys=['indexi', 'indexl', 'indexas'])
print(len(df.index))

1113578


# drop duplicate asid

In [7]:
df = df.drop_duplicates('author_string_id')

In [8]:
df.head()

author_string_id first_initial forenames  \
indexi indexl     indexas                                               
z      zytkiewicz 440967.0           440967.0             z  zbigniew   
       zyambo     684583.0           684583.0             z      zude   
       zy         159843.0           159843.0             z       zou   
       zuo        1137329.0         1137329.0             z   zonglin   
                  661540.0           661540.0             z  zong-yan   

                            initials        journal_format  journal_id  \
indexi indexl     indexas                                                
z      zytkiewicz 440967.0       z r  names_initials_names      1254.0   
       zyambo     684583.0         z            only_names       352.0   
       zy         159843.0         z            only_names       441.0   
       zuo        1137329.0        z            only_names       110.0   
                  661540.0         z            only_names        31.0   

                              last_name  last_names              norm_name  \
indexi indexl     indexas                                                    
z      zytkiewicz 440967.0   zytkiewicz  zytkiewicz  zbigniew r zytkiewicz   
       zyambo     684583.0       zyambo      zyambo            zude zyambo   
       zy         159843.0           zy          zy                 zou zy   
       zuo        1137329.0         zuo         zuo            zonglin zuo   
                  661540.0          zuo         zuo           zong-yan zuo   

                             paper_id                raw_name  
indexi indexl     indexas                                      
z      zytkiewicz 440967.0   148900.0  Zbigniew R. Zytkiewicz  
       zyambo     684583.0   260716.0             Zude Zyambo  
       zy         159843.0    46355.0                  Zou ZY  
       zuo        1137329.0  523268.0             Zonglin Zuo  
                  661540.0   250838.0            Zong-Yan Zuo

In [9]:
len(df)

1113578

In [10]:
prefix_list = list(pd.read_csv('name_prefixes.csv', header=None, encoding = 'ISO-8859-1')[0]) 
known_error_list = list(pd.read_csv('error_match.csv', header=None, encoding = 'ISO-8859-1')[0]) 

In [11]:
# raw = 'Mark Jones'
# norm = util.normalise_name(raw)
# forenames, initials, last_names, name_format = util.split_name(norm, raw)
# util.get_names_regex(forenames, initials, last_names)

# test single name

In [12]:
raw = 'mark jones'
util.match(raw, df, prefix_list, known_error_list)

author_string_id first_initial forenames initials  \
indexi indexl indexas                                                       
m      jones  906572.0          906572.0             m      mark      m r   
              111785.0          111785.0             m      mark        m   
              323721.0          323721.0             m                m r   
              672241.0          672241.0             m              m p a   
              726564.0          726564.0             m                m l   
              424369.0          424369.0             m                m l   
              652290.0          652290.0             m                m k   
              488551.0          488551.0             m                m k   
              797177.0          797177.0             m                m j   
              726702.0          726702.0             m                m i   
              465447.0          465447.0             m                m i   
              749412.0          749412.0             m                m d   
              737106.0          737106.0             m                m d   
              158903.0          158903.0             m                  m   
              139258.0          139258.0             m                  m   

                              journal_format  journal_id last_name last_names  \
indexi indexl indexas                                                           
m      jones  906572.0  names_initials_names      1850.0     jones      jones   
              111785.0            only_names      1261.0     jones      jones   
              323721.0        initials_names       905.0     jones      jones   
              672241.0        initials_names      1230.0     jones      jones   
              726564.0        initials_names      1203.0     jones      jones   
              424369.0        initials_names      1203.0     jones      jones   
              652290.0        initials_names      1228.0     jones      jones   
              488551.0        initials_names      1228.0     jones      jones   
              797177.0        initials_names       799.0     jones      jones   
              726702.0        initials_names      1253.0     jones      jones   
              465447.0        initials_names      1253.0     jones      jones   
              749412.0        initials_names      1228.0     jones      jones   
              737106.0        initials_names      1439.0     jones      jones   
              158903.0        names_initials       438.0     jones      jones   
              139258.0        initials_names      1228.0     jones      jones   

                           norm_name  paper_id        raw_name  
indexi indexl indexas                                           
m      jones  906572.0  mark r jones  440415.0   Mark R. Jones  
              111785.0    mark jones  511307.0      Mark Jones  
              323721.0     m r jones  103934.0     M. R. Jones  
              672241.0   m p a jones  255157.0  M. P. A. Jones  
              726564.0     m l jones  280328.0      M.L. Jones  
              424369.0     m l jones  247103.0     M. L. Jones  
              652290.0     m k jones  245090.0      M.K. Jones  
              488551.0     m k jones  221799.0     M. K. Jones  
              797177.0     m j jones  317484.0       M J Jones  
              726702.0     m i jones  280967.0     M. I. Jones  
              465447.0     m i jones  159014.0      M.I. Jones  
              749412.0     m d jones  291318.0      M.D. Jones  
              737106.0     m d jones  534947.0     M. D. Jones  
              158903.0       jones m   46205.0         Jones M  
              139258.0       m jones  178090.0        M. Jones

# write the author variants to databse

In [15]:
def append_dictionary_author_variants(df):
    
    config = { 
        "dbname":"testdb",
        "user": "postgres",
        "host": "localhost",
        "dbpass": "aberdare"
    }
    
    engine = create_engine('postgresql://{user}:{dbpass}@{host}/{dbname}'.format(
        user=config['user'],
        dbpass=config['dbpass'],
        host=config['host'],
        dbname=config['dbname']
    ))
    

    imax = 401 # debugging break point
    
    total = len(df.index) # total lines to read  
    total_output = 1      # track the number of characters in output file, hack to remove last comma from json    
    chunk_size = 100      # number of keys to write to table per chunk
    
    asid = None           # author string id
    first_initial = None
    last_name = None
        
    i = 0            # track row
    
    
    
    # loop through dataframe
    for index in df.index:
        
        try: ## hack deal with exceptions
        
            asid = df.at[index, 'author_string_id'] # get author string id

            # reduce the size of the target dataframe to compare to rows where the first initial and last name match
            if index[:2] != (first_initial, last_name): # only upate the target data frame if the first inital or last name have changed

                # update first initial and last name
                first_initial = df.at[index, 'first_initial'] 
                last_name = df.at[index, 'last_name']

                # update target data frame
                target = df.loc[(df.first_initial.values==first_initial)&(df.last_name.values==last_name)]
                #print('target len {}'.format(len(target)))

            # get forename, initials and last names 
            forenames = df.at[index, 'forenames']        
            initials = df.at[index, 'initials']
            last_names = df.at[index, 'last_names']

            # get variants

            res = util.match_variants(forenames, initials, last_names, target)
            if len(res) > 1:
                res['asid2'] = res.author_string_id
                res['asid1'] = asid
                #print(res[['asid1', 'asid2']])
                q.append(res[['asid1', 'asid2']], 'author_variant')
                #res[['asid1', 'asid2']].to_sql('author_variant', engine, index=False, if_exists='append')

    #         if len(res.index) > 1: #ignore matches with itself
    #             #dasid[str(asid)] = list(res.loc[res.author_string_id!=asid].author_string_id)
    #             output_string += '"{}": {}, '.format(str(asid), str(list(res.author_string_id)))
    
        except:
            pass
            
        i+=1
        #print('row {}'.format(i))
            

            
        if (i % chunk_size) == 0:
            
            clear_output(wait=True) # update progress
            print('row {}, {:.1f}%'.format(i, 100.*i/total))
            
            
        if i > imax:
            return             
    

    return 
    
    
t0 = df.loc[df.norm_name.isin(['mark r jones', 'mark jones', 'm jones', 'm r jones',  
       'm d jones'])]

t1 = df[df.paper_id.isin(list(t0.paper_id))]

#t = df.loc[df.norm.isin(['z zou', 'z y zou'])]

print('start')
%prun -l 8 append_dictionary_author_variants(df)
print('done')


row 400, 0.0%
 done


In [ ]:
# def get_dictionary_author_variants(df, output_filename):
    
#     text_file = open(output_filename, "a") # open the output file
#     text_file.truncate(0)                  # clear the file
#     text_file.write('{')                   # open json object

#     #imax = 1000 # debugging break point
    
#     total = len(df.index) # total line to red  
#     total_output = 1      # track the number of characters in output file, hack to remove last comma from json    
#     chunk_size = 100      # number of keys to write to file per chunk
    
#     asid = None           # author string id
#     first_initial = None
#     last_name = None
        
#     i = 0            # track row
    
#     output_string = ''    
    
#     # loop through dataframe
#     for index in df.index:
        
#         asid = df.at[index, 'author_string_id'] # get author string id
        
#         # reduce the size of the target dataframe to compare to rows where the first initial and last name match
#         if index[:2] != (first_initial, last_name): # only upate the target data frame if the first inital or last name have changed
        
#             # update first initial and last name
#             first_initial = df.at[index, 'first_initial'] 
#             last_name = df.at[index, 'last_name']
            
#             # update target data frame
#             target = df.loc[(df.first_initial.values==first_initial)&(df.last_name.values==last_name)]
#             #print('target len {}'.format(len(target)))
           
#         # get forename, initials and last names 
#         forenames = df.at[index, 'forenames']        
#         initials = df.at[index, 'initials']
#         last_names = df.at[index, 'last_names']
        
#         # get variants
#         res = util.match_variants(forenames, initials, last_names, target)
        
#         if len(res.index) > 1: #ignore matches with itself
#             #dasid[str(asid)] = list(res.loc[res.author_string_id!=asid].author_string_id)
#             output_string += '"{}": {}, '.format(str(asid), str(list(res.author_string_id)))
            
#         i+=1
#         print('row {}'.format(i))
            

            
#         if (i % chunk_size) == 0:
#             text_file.write(output_string) # write to text file
#             total_output += len(output_string) # track how many characters have been added
#             #print(output_string)
#             output_string = '' # restart next chunk with blank output string
#             clear_output(wait=True) # update progress
#             print('row {}, {:.1f}%'.format(i, 100.*i/total))
            
            
# #         if i > imax:
# #             text_file.write(output_string) # write to text file
# #             total_output += len(output_string) # track how many characters have been added
# #             text_file.truncate(total_output-2) # remove the last ", " from file
# #             text_file.write('}') # close the output json
# #             text_file.close()
# #             return             
    
#     text_file.write(output_string) # write to text file
#     total_output += len(output_string) # track how many characters have been added
#     text_file.truncate(total_output-2) # remove the last ", " from file
#     text_file.write('}') # close the output json
#     text_file.close()

#     return 
    
    
# t0 = df.loc[df.norm_name.isin(['mark r jones', 'mark jones', 'm jones', 'm r jones',  
#        'm d jones'])]

# t1 = df[df.paper_id.isin(list(t0.paper_id))]

# #t = df.loc[df.norm.isin(['z zou', 'z y zou'])]

# print('start')
# get_dictionary_author_variants(df.iloc[5200:], 'author_variants_lookup.json')
# print('done')


In [ ]:
# df.iloc[5200:5300]

In [ ]:
# t0 = df.loc[df.norm_name.isin(['mark r jones', 'mark jones', 'm jones', 'm r jones',  
#        'm d jones'])]

# t1 = df[df.paper_id.isin(list(t0.paper_id))]

# get_dictionary_author_variants(t1, 'test_author_variants.json')
# author_variants = json.load(open('test_author_variants.json'))
# author_variants